In [1]:
import React from 'react';
import Highcharts from 'highcharts/highstock';
import HighchartsReact from 'highcharts-react-official';

const StockChart = ({ data }) => {
  const options = {
    title: {
      text: 'Stock Price'
    },
    xAxis: {
      type: 'datetime'
    },
    yAxis: {
      title: {
        text: 'Price'
      }
    },
    series: [{
      name: 'Stock Price',
      data: data,
      tooltip: {
        valueDecimals: 2
      }
    }]
  };

  return (
    <HighchartsReact
      highcharts={Highcharts}
      constructorType={'stockChart'}
      options={options}
    />
  );
};

export default StockChart;

SyntaxError: invalid syntax (2490503976.py, line 1)

In [ ]:
 if (isChatActive) {
    return (
      <div className={styles.chatPage}>
        <div className={styles.chatHeader}>
          <div className={styles.chatHeaderTitle}>
            <div className={styles.iconWrapper}>
              <FluentIcons.Sparkle24Filled />
            </div>
            <Text className={styles.chatHeaderText}>{userConfig.buttons[0].label}</Text>
          </div>
          <Text className={styles.chatHeaderDescription}>{userConfig.buttons[0].description}</Text>
        </div>
        <div className={styles.chatMessages}>
          {chatMessages.map((message, index) => (
            <React.Fragment key={index}>
              <div
                className={`${styles.chatMessage} ${
                  message.role === "user" ? styles.userMessage : styles.assistantMessage
                }`}
              >
                {message.role === "user" ? (
                  message.content
                ) : (
                  <div dangerouslySetInnerHTML={{ __html: message.content }} />
                )}
              </div>
              {message.role === "user" && index === 0 && isFirstQuery && (
                <div className={styles.chartContainer}>
                  <StockChart data={stockData} />
                </div>
              )}
              {message.subgraphResults && (
                <div className={styles.subgraphMessage}>
                  <SubgraphResults results={message.subgraphResults} />
                </div>
              )}
            </React.Fragment>
          ))}
        </div>

In [ ]:
from typing import Dict, List
from langgraph.graph import StateGraph, END

# Define your functions
def vector_db_search(query: str) -> str:
    # Implement your vector database search here
    return f"Vector DB result for: {query}"

def query_expansion(query: str) -> List[str]:
    # Implement your query expansion logic here
    return [f"{query} expanded 1", f"{query} expanded 2"]

def api_call(query: str) -> str:
    # Implement your API call here
    return f"API result for: {query}"

# Define the routing logic
def route_query(state):
    query = state['query']
    if len(query.split()) > 3:  # Simple routing logic, adjust as needed
        return "query_expansion"
    else:
        return "vector_db_search"

# Define the query expansion and API call logic
def expand_and_call(state):
    expanded_queries = query_expansion(state['query'])
    results = []
    for eq in expanded_queries:
        results.append(api_call(eq))
    return {"expanded_results": results}

# Define the aggregation logic
def aggregate_results(state):
    if "expanded_results" in state:
        # Aggregate expanded results
        combined = " ".join(state["expanded_results"])
        return {"response": f"Aggregated results: {combined}"}
    else:
        # Return vector DB result
        return {"response": state["result"]}

# Create the graph
workflow = StateGraph(name="QueryProcessor")

# Add nodes
workflow.add_node("route", route_query)
workflow.add_node("vector_db_search", lambda state: {"result": vector_db_search(state['query'])})
workflow.add_node("expand_and_call", expand_and_call)
workflow.add_node("aggregate", aggregate_results)

# Add edges
workflow.add_edge("route", "vector_db_search")
workflow.add_edge("route", "expand_and_call")
workflow.add_edge("vector_db_search", "aggregate")
workflow.add_edge("expand_and_call", "aggregate")
workflow.add_edge("aggregate", END)

# Set the entrypoint
workflow.set_entry_point("route")

# Compile the graph
app = workflow.compile()

# Example usage
query = "What is the capital of France?"
result = app.invoke({"query": query})
print(result['response'])

In [ ]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

# Check if CUDA is available, otherwise fallback to CPU (GPU is prioritized)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set up the original training data (noisy sine wave) and move to GPU
train_x_full = torch.linspace(0, 1, 100).cuda()  # 100 data points
train_y_full = (torch.sin(train_x_full * (2 * math.pi)) + 
                torch.randn(train_x_full.size()).cuda() * math.sqrt(0.04))

# Define the GP Model (with device support)
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean().cuda()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel().cuda()).cuda()

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Initialize likelihood and move it to GPU
likelihood = gpytorch.likelihoods.GaussianLikelihood().cuda()

# Incrementally add data and retrain the model before predicting the next day
results = []

initial_train_size = 10  # Start with 10 initial points
for day in range(initial_train_size, len(train_x_full)):
    # Add one more data point to the training set
    train_x = train_x_full[:day]
    train_y = train_y_full[:day]
    test_x = train_x_full[day].unsqueeze(0)  # The next data point to predict

    # Define the GP model with the updated training set and move to GPU
    model = ExactGPModel(train_x, train_y, likelihood).cuda()

    # Set training mode
    model.train()
    likelihood.train()

    # Use the Adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    # Training loop
    training_iter = 50
    for i in range(training_iter):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        optimizer.step()

    # Switch to evaluation mode after training
    model.eval()
    likelihood.eval()

    # Predict on the next day (test_x)
    with torch.no_grad():
        observed_pred = likelihood(model(test_x))

    # Collect the prediction and actual result
    pred_mean = observed_pred.mean.cpu().numpy()  # Move prediction to CPU for plotting
    test_actual = train_y_full[day].cpu().numpy()  # Actual value for comparison

    results.append({
        "day": day,
        "test_actual": test_actual,
        "pred_mean": pred_mean
    })

# Visualize the results
days = [result["day"] for result in results]
test_actuals = [result["test_actual"] for result in results]
pred_means = [result["pred_mean"] for result in results]

plt.plot(days, test_actuals, 'k*', label="Actual")
plt.plot(days, pred_means, 'b', label="Predicted Mean")
plt.fill_between(
    days, 
    [result["pred_mean"] - 2*math.sqrt(0.04) for result in results], 
    [result["pred_mean"] + 2*math.sqrt(0.04) for result in results], 
    color='blue', alpha=0.5
)
plt.legend()
plt.xlabel('Days')
plt.ylabel('Values')
plt.title('Incremental GP Predictions with Daily Data Addition (Cross-Validation)')
plt.show()
